In [1]:
import os.path
from tqdm import tqdm
import torch
import torch.nn.functional as F
from dataset_mixed import *  # dataset
from model import *  # actual MIL model
from sklearn import metrics as metrics
import csv
import os
import shutil
import pandas as pd
import re

In [218]:
CLASSES = ['control', 'RUNX1_RUNX1T1', 'NPM1', 'CBFB_MYH11', 'PML_RARA']
num_classes = 5
seed=42
experiment_source=f'experiment_3_seed{seed}'
real_data_source='/mnt/volume/shared/data_file/data'
SOURCE_FOLDER = '/mnt/volume/shared/data_file/artificialdata/'+experiment_source
TARGET_FOLDER = '/mnt/volume/shared/exp4/dropout_original/result_folder_1'
output_folder = f'/mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed{seed}'

def get_patient_name(path):
    return re.search(r"/data/\w+/([0-9a-zA-Z_]*)", path).group(1)

def get_class_name(path):
    return re.search(r"/data/(\w+)", path).group(1)

In [219]:
patients = {}
with open(os.path.join(SOURCE_FOLDER,'metadata.csv'), newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)
    idx = 0
    for line in reader:
        key = line[0] + "_" + line[3]
        patients[key] = [os.path.join(SOURCE_FOLDER,
                                         "data",
                                         line[3],
                                         line[0],
                                         "fnl34_bn_features_layer_7.npy"), line[3]]

In [220]:
def update_misclassification_count(probability_vector, one_hot_target, current_misclassification_count):
    """
    Update misclassification count based on the given probability vector and one-hot encoded target vector.

    Parameters:
    - probability_vector: Tensor representing the probability vector.
    - one_hot_target: Tensor representing the one-hot encoded target vector.
    - current_misclassification_count: Current misclassification count.

    Returns:
    - Updated misclassification count.
    """
    #print("Target: ", one_hot_target)
    #print(probability_vector)
    # Convert probability vector to one-hot encoded vector
    one_hot_prediction = torch.zeros_like(probability_vector)
    one_hot_prediction[0, torch.argmax(probability_vector).item()] = 1
    #print("One hot prediction is: ", one_hot_prediction)

    # Find the index where the target is 1
    target_index = torch.argmax(one_hot_target).item()
    #print("Target index is :", target_index)

    # Check for misclassification and update count
    if torch.argmax(one_hot_prediction).item() != target_index:
        current_misclassification_count += 1

    return current_misclassification_count

In [221]:
# Number of Monte Carlo samples
num_samples = 10

class_converter = {}

with open(os.path.join(TARGET_FOLDER,'class_conversion.csv'), newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)
    for line in reader:
        class_converter[line[1]] = int(line[0])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# load best performing model, and launch on test set
model = torch.load(os.path.join(TARGET_FOLDER, "state_dictmodel.pt"),map_location="cpu")
model = model.to(device)
model.train()

# Initialize arrays to store uncertainties
all_uncertainties = {}
missclassification_counts = {}
max_uncertainties = {}
sum_uncertainties = {}

# Perform Monte Carlo Dropout
with torch.no_grad():
    for p in patients.keys():
        # Make predictions with dropout enabled multiple times
        pred = []
        missclassification_count = 0
        for j in range(num_samples):
            path, lbl_name = patients[p]
            lbl = np.zeros(5)
            lbl[class_converter[lbl_name]] = 1

            bag = np.load(path)

            bag = torch.tensor(bag).to(device)
            bag = torch.unsqueeze(bag,0)
            prediction, _, _, _,  = model(bag)
            pred.append(F.softmax(prediction,dim=1).cpu().detach().numpy())

            # Class missclassification count update
            missclassification_count = update_misclassification_count(F.softmax(prediction,dim=1), torch.tensor(lbl), missclassification_count)

        # Convert the list of tensors to a PyTorch tensor
        pred_tensor = [torch.from_numpy(arr) for arr in pred]
        pred_tensor = torch.stack(pred_tensor)
        
        # Compute mean and uncertainty
        mean_prediction = pred_tensor.mean(dim=0)
        #print('Mean prediction is:', mean_prediction)
        
        uncertainty = pred_tensor.std(dim=0)
        #print('Uncertainty tensor is:', uncertainty)
        
        uncertainty_value_max = torch.max(uncertainty).item()
        uncertainty_value_sum = torch.sum(uncertainty).item()
        path=os.path.dirname(path)
        # Save uncertainty results along with the path
        max_uncertainties[p] = {'path': path, 'data': uncertainty.cpu().numpy().squeeze(), 'uncertainty': uncertainty_value_max}
        sum_uncertainties[p] = {'path': path, 'data': uncertainty.cpu().numpy().squeeze(), 'uncertainty': uncertainty_value_sum}
        missclassification_counts[p] = {'path': path, 'uncertainty' : missclassification_count / num_samples}

In [222]:
def sort_and_print(uncertainties):
    # Sort misclassification counts dictionary by values (highest to lowest)
    sorted_uncertainties = dict(sorted(uncertainties.items(), key=lambda item: item[1]['uncertainty'], reverse=True))
    
    # Print sorted misclassification counts
    for p, data in sorted_uncertainties.items():
        uncertainty_rate = data['uncertainty']
        print(f"Patient {p}: Uncertainty - {uncertainty_rate:.4}")


In [223]:
sort_and_print(max_uncertainties)
#sort_and_print(sum_uncertainties)
#sort_and_print(missclassification_counts)

Patient patient_4_PML_RARA: Uncertainty - 0.2373
Patient patient_32_NPM1: Uncertainty - 0.1995
Patient patient_19_PML_RARA: Uncertainty - 0.1925
Patient patient_6_NPM1: Uncertainty - 0.1898
Patient patient_10_PML_RARA: Uncertainty - 0.1897
Patient patient_11_NPM1: Uncertainty - 0.1793
Patient patient_39_PML_RARA: Uncertainty - 0.1773
Patient patient_8_NPM1: Uncertainty - 0.1757
Patient patient_30_CBFB_MYH11: Uncertainty - 0.1754
Patient patient_11_PML_RARA: Uncertainty - 0.1752
Patient patient_28_NPM1: Uncertainty - 0.1714
Patient patient_28_control: Uncertainty - 0.1683
Patient patient_37_PML_RARA: Uncertainty - 0.1672
Patient patient_8_RUNX1_RUNX1T1: Uncertainty - 0.1665
Patient patient_35_RUNX1_RUNX1T1: Uncertainty - 0.1659
Patient patient_9_PML_RARA: Uncertainty - 0.1635
Patient patient_29_RUNX1_RUNX1T1: Uncertainty - 0.1628
Patient patient_19_NPM1: Uncertainty - 0.1621
Patient patient_15_CBFB_MYH11: Uncertainty - 0.1619
Patient patient_26_CBFB_MYH11: Uncertainty - 0.1582
Patient p

In [224]:
print(len(max_uncertainties.keys()))
print(len(set(max_uncertainties.keys())))

205
205


In [225]:
def select_paths(uncertainties, percentage):
    sorted_uncertainties = dict(sorted(uncertainties.items(), key=lambda item: item[1]['uncertainty'], reverse=True))
    
    # Calculate the number of paths based on the percentage
    num_paths = int(len(sorted_uncertainties) * (percentage / 100.0))
    
    # Select the top percentage of paths
    selected_paths = {p: data['path'] for p, data in list(sorted_uncertainties.items())[:num_paths]}
    return selected_paths

metadata_path = SOURCE_FOLDER + "/metadata.csv"

def create_metadata(selected_paths, target_folder):
    # Load the existing metadata CSV file into a DataFrame
    metadata_df = pd.read_csv(metadata_path)

    # Dictionary to store the extracted information
    metadata_dict = {}

    # Loop over each key in selected_paths
    for key in selected_paths:
        path = selected_paths[key]

        # Extract patient_id and bag_label from the key
        match = re.search(r'\d', key)
        if match:
            index_of_second_digit = key.find('_', match.start() + 1)
            if index_of_second_digit != -1:
                patient_id = key[:index_of_second_digit + 1]  # Include everything up to the second underscore
                patient_id = patient_id[:-1]
                bag_label = key[index_of_second_digit + 1:]  # Remove the first underscore in bag_label
            else:
                # If no second underscore found, consider the entire key as patient_id
                patient_id = key
                bag_label = ''
        else:
            # If no digit found, consider the entire key as patient_id
            patient_id = key
            bag_label = ''

        # Filter the metadata DataFrame based on patient_id and bag_label
        metadata_row = metadata_df[(metadata_df['patient_id'] == patient_id) & (metadata_df['bag_label'] == bag_label)]

        # Check if any matching rows are found
        if not metadata_row.empty:
            # Get the first matching row (assuming there's only one match)
            metadata_dict[key] = metadata_row.iloc[0].to_dict()
        else:
            # Handle the case when no matching rows are found
            print(f"No metadata found for {key}: {path}")

    # Create a new DataFrame from the extracted metadata dictionary
    new_metadata_df = pd.DataFrame.from_dict(metadata_dict, orient='index')


    # Save the new metadata DataFrame to a CSV file in the target folder
    new_metadata_path = os.path.join(target_folder, 'metadata_uncertain_patients.csv')
    new_metadata_df.to_csv(new_metadata_path, index=False)

    return new_metadata_path


#returns dict that contains number of real patients per leukemia class
def get_realpatients_filepaths_dictionary(src_folder):
    paths = {}
    for class_name in os.listdir(src_folder):
        class_path=os.path.join(src_folder, class_name)
        for file in  os.listdir(class_path):
            src_path = os.path.join(class_path, file)
            #print(src_path)
            # Extract class and patient information from the source path
            patient_name = get_patient_name(src_path)
            if class_name not in paths.keys():
                paths[class_name] = []
            paths[class_name].append(src_path)      
    return paths

In [226]:
import copy
def save_patient_filepaths(selected_paths, new_folder,paths_real_patients):
    # save filepaths to the new folder
    print(f"Save file paths for uncertain patients in {new_folder}")
    os.makedirs(new_folder, exist_ok=True)
    # save filepath in dictionary paths with keys as class and values list of file paths
    paths_uncertain_patients = {}
    #Initilialise 
    for class_name in CLASSES:
        if class_name not in paths_uncertain_patients.keys():
                    paths_uncertain_patients[class_name] = []
            
    for p, path in selected_paths.items():   
        class_name = get_class_name(path)
        paths_uncertain_patients[class_name].append(path)    
        #print(class_name, path)
    
    #Concatenate the two dictionaries
    paths_mixed_patients = copy.deepcopy(paths_real_patients)
    for key, value in paths_mixed_patients.items():
        paths_mixed_patients[key] += paths_uncertain_patients[key]
        #print(key,  paths_mixed_patients[key])
    #print(paths_mixed_patients)   
    #Delete duplicates from list:
    for key in paths_mixed_patients.keys():
        len_before = len(paths_mixed_patients[key])
        paths_mixed_patients[key] = list(set(paths_mixed_patients[key]))
        print(f"Removed {len_before -len(paths_mixed_patients[key])} duplicates")
    #Save ditionary
    with open(new_folder+'/file_paths.pkl', 'wb') as f:
        pickle.dump(paths_mixed_patients, f)


In [227]:
# Iterate over different percentages from 10 to 50 and save uncertain patients
paths_real_patients=get_realpatients_filepaths_dictionary(real_data_source)
for percentage in [10,20,30,50]:
    # Define folder to save the uncertain patients
    new_folder_max = output_folder+f'/max_{percentage}_percent'
    
    # Select uncertain paths based on the current percentage for max_uncertainties
    selected_max_paths = select_paths(max_uncertainties, percentage)
    # Save uncertain patients to the new folder
    save_patient_filepaths(selected_max_paths, new_folder_max,paths_real_patients)
    create_metadata(selected_max_paths, new_folder_max)

Save file paths for uncertain patients in /mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed42/max_10_percent
Removed 0 duplicates
Removed 0 duplicates
Removed 0 duplicates
Removed 0 duplicates
Removed 0 duplicates
Save file paths for uncertain patients in /mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed42/max_20_percent
Removed 0 duplicates
Removed 0 duplicates
Removed 0 duplicates
Removed 0 duplicates
Removed 0 duplicates
Save file paths for uncertain patients in /mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed42/max_30_percent
Removed 0 duplicates
Removed 0 duplicates
Removed 0 duplicates
Removed 0 duplicates
Removed 0 duplicates
Save file paths for uncertain patients in /mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed42/max_50_percent
Removed 0 duplicates
Removed 0 duplicates
Removed 0 duplicates
Removed 0 duplicates
Removed 0 duplicates


In [228]:

def concatenate_metadata(original_metadata_path,uncertain_patients_folder):
    # Load the original metadata CSV file
    original_metadata_df = pd.read_csv(original_metadata_path)

    # Loop over folders in uncertain_patients
    for folder in os.listdir(uncertain_patients_folder):
        folder_path = os.path.join(uncertain_patients_folder, folder)
        #print(folder_path)

        # Check if the item in the uncertain_patients_folder is a directory
        if os.path.isdir(folder_path):
            # Look for metadata.csv in each folder
            metadata_file_path = os.path.join(folder_path, 'metadata_uncertain_patients.csv')
            print("metadata is:")
            print(metadata_file_path)

            if os.path.exists(metadata_file_path):
                # Load the metadata from the current folder
                uncertain_metadata_df = pd.read_csv(metadata_file_path)

                # Concatenate with the original metadata
                concatenated_metadata_df = pd.concat([original_metadata_df, uncertain_metadata_df], ignore_index=True)
                # Save the concatenated metadata to a new CSV file in the output folder
                output_folder_new = uncertain_patients_folder + f'/{folder}'
                os.makedirs(output_folder_new, exist_ok=True)
                output_file_path = os.path.join(output_folder_new, 'metadata.csv')
                print("Output is:")
                print(output_file_path)
                concatenated_metadata_df.to_csv(output_file_path, index=False)

In [229]:
original_metadata_path="/mnt/volume/shared/data_file/metadata.csv"
concatenate_metadata(original_metadata_path,output_folder)

metadata is:
/mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed42/max_20_percent/metadata_uncertain_patients.csv
Output is:
/mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed42/max_20_percent/metadata.csv
metadata is:
/mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed42/max_50_percent/metadata_uncertain_patients.csv
Output is:
/mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed42/max_50_percent/metadata.csv
metadata is:
/mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed42/max_30_percent/metadata_uncertain_patients.csv
Output is:
/mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed42/max_30_percent/metadata.csv
metadata is:
/mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed42/max_10_percent/metadata_uncertain_patients.csv
Output is:
/mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed42/max_10_percent/metadata.csv


In [230]:
import pickle
seed=1
percentage=20
folder = f'/mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed{seed}/max_{percentage}_percent'
with open(folder+'/file_paths.pkl','rb') as f:
    m = pickle.load(f)  

In [231]:
m

{'control': ['/mnt/volume/shared/data_file/data/control/CIQ',
  '/mnt/volume/shared/data_file/data/control/CCO',
  '/mnt/volume/shared/data_file/data/control/MPP',
  '/mnt/volume/shared/data_file/data/control/YWQ',
  '/mnt/volume/shared/data_file/data/control/NOL',
  '/mnt/volume/shared/data_file/data/control/VPN',
  '/mnt/volume/shared/data_file/data/control/TVZ',
  '/mnt/volume/shared/data_file/data/control/LAM',
  '/mnt/volume/shared/data_file/data/control/GJQ',
  '/mnt/volume/shared/data_file/data/control/FAQ',
  '/mnt/volume/shared/data_file/data/control/CDA',
  '/mnt/volume/shared/data_file/data/control/AIH',
  '/mnt/volume/shared/data_file/data/control/OTU',
  '/mnt/volume/shared/data_file/data/control/ORD',
  '/mnt/volume/shared/data_file/data/control/ATM',
  '/mnt/volume/shared/data_file/data/control/FDW',
  '/mnt/volume/shared/data_file/data/control/UNX',
  '/mnt/volume/shared/data_file/data/control/ZNL',
  '/mnt/volume/shared/data_file/data/control/DHE',
  '/mnt/volume/share

In [233]:
#Check for duplicates
seeds = [1, 20, 42]
percentages = [10, 20, 30, 50]

for seed in seeds:
    for percentage in percentages:
        folder = f'/mnt/volume/shared/data_file/mixed_uncertain_fixbug_seed{seed}/max_{percentage}_percent'
        
        with open(folder + '/file_paths.pkl', 'rb') as f:
            file_paths_dict = pickle.load(f)

        # Check for duplicates within each list
        for key, value_list in file_paths_dict.items():
            if len(value_list) != len(set(value_list)):
                print(f"Duplicates found for seed {seed}, {percentage}%, and key {key}. Duplicates: {set([item for item in value_list if value_list.count(item) > 1])}")
